In [1]:
import json
import os

import pandas as pd

/var/folders/xg/5ns375k927gfzbg4c143d4zr0000gn/T/ipykernel_18791/1943265658.py:4: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
with open('./data/ml_energy/models.json', 'r') as fd:
    model_params = json.load(fd)

In [3]:
all_dfs = []

for root, _, files in os.walk("./data/ml_energy"):
    for file in files:
        if 'benchmark' in file and file.endswith('.csv'):
            gpu, task, _ = file.split('_')
            df = pd.read_csv(os.path.join(root, file))
            df['gpu'] = gpu
            df['task'] = task
            df['parameters_count'] = df.model.apply(lambda x: model_params[x]['params'])
            all_dfs.append(df)

df = pd.concat(all_dfs)

In [4]:
df['energy_per_token'] = (df['energy'] / df['response_length']) / 3600

In [5]:
df.to_csv('./data/aggregated_ml_energy.csv', index=False)